In [1]:
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf
tf.__version__


'1.12.0'

In [12]:
import pandas as pd
import time
import os
import numpy as np
import scipy as sp
import re
from sklearn.feature_extraction import stop_words as stop
from sklearn.metrics import roc_auc_score as auroc
from sklearn.metrics import confusion_matrix as cm
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 10)

In [6]:
data = pd.read_csv("./Data/DJIA_table.csv")
data.head()


,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234


In [35]:
data2 = pd.read_csv("./Data/Combined_News_DJIA.csv")
data2.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as countries move to brink of war""",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)',"b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'","b""Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing""",b'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.',"b""Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side""","b""The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it.""",...,"b'Georgia Invades South Ossetia - if Russia gets involved, will NATO absorb Georgia and unleash a full scale war?'",b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to prevent an Israeli strike on Iran."" Israeli Defense Minister Ehud Barak: ""Israel is prepared for uncompromising victory in the case of military hostilities.""'",b'This is a busy day: The European Union has approved new sanctions against Iran in protest at its nuclear programme.',"b""Georgia will withdraw 1,000 soldiers from Iraq to help fight off Russian forces in Georgia's breakaway region of South Ossetia""",b'Why the Pentagon Thinks Attacking Iran is a Bad Idea - US News &amp; World Report',b'Caucasus in crisis: Georgia invades South Ossetia',"b'Indian shoe manufactory - And again in a series of ""you do not like your work?""'",b'Visitors Suffering from Mental Illnesses Banned from Olympics',"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,"b'Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq?'",b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli training, we're fending off Russia """,b'Georgian army flees in disarray as Russians advance - Gori abandoned to Russia without a shot fired',"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zealand Passports doing in Iraq?',b'Russia angered by Israeli military sale to Georgia',b'An American citizen living in S.Ossetia blames U.S. and Georgian leaders for the genocide of innocent people',...,b'Israel and the US behind the Georgian aggression?',"b'""Do not believe TV, neither Russian nor Georgian. There are much more victims""'",b'Riots are still going on in Montreal (Canada) because police murdered a boy on Saturday.',b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Torture',b' Russia has just beaten the United States over the head with Peak Oil',b'Perhaps *the* question about the Georgia - Russia conflict ',b'Russia is so much better at war',"b""So this is what it's come to: trading sex for food."""
2,2008-08-12,0,"b'Remember that adorable 9-year-old who sang at the opening ceremonies? That was fake, too.'","b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would have no children...""'","b""Al-Qa'eda is losing support in Iraq because of a brutal crackdown on activities it regards as un-Islamic - including women buying cucumbers""",b'Ceasefire in Georgia: Putin Outmaneuvers the West',b'Why Microsoft and Intel tried to kill the XO $100 laptop',b'Stratfor: The Russo-Georgian War and the Balance of Power ',"b""I'm Trying to Get a Sense of This Whole Georgia-Russia War: Vote Up If You Think Georgia Started It, Or Down If you Think Russia Did""",...,b'U.S. troops still in Georgia (did you know they were in Georgia in the first place?)',b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. 

In [36]:
def df_regx_replace(df, columns, regx):
    for col in columns:
        df[col] = df[col].replace(to_replace = regx, value = " ", regex = True)
        df[col] = df[col].str.lower()
    return df

columns = data2.iloc[:, 2:]
#print(columns)
regx = "[\s\W*|\W\s*]"

data2 = df_regx_replace(data2, columns, regx)
data2.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,b georgia downs two russian warplanes as countries move to brink of war,b breaking musharraf to be impeached,b russia today columns of troops roll into south ossetia footage from fighting youtube,b russian tanks are moving towards the capital of south ossetia which has reportedly been completely destroyed by georgian artillery fire,b afghan children raped with impunity u n official says this is sick a three year old was raped and they do nothing,b 150 russian tanks have entered south ossetia whilst georgia shoots down two russian jets,b breaking georgia invades south ossetia russia warned it would intervene on so s side,b the enemy combatent trials are nothing but a sham salim haman has been sentenced to 5 1 2 years but will be kept longer anyway just because they feel like it,...,b georgia invades south ossetia if russia gets involved will nato absorb georgia and unleash a full scale war,b al qaeda faces islamist backlash,b condoleezza rice the us would not act to prevent an israeli strike on iran israeli defense minister ehud barak israel is prepared for uncompromising victory in the case of military hostilities,b this is a busy day the european union has approved new sanctions against iran in protest at its nuclear programme,b georgia will withdraw 1 000 soldiers from iraq to help fight off russian forces in georgia s breakaway region of south ossetia,b why the pentagon thinks attacking iran is a bad idea us news amp world report,b caucasus in crisis georgia invades south ossetia,b indian shoe manufactory and again in a series of you do not like your work,b visitors suffering from mental illnesses banned from olympics,b no help for mexico s kidnapping surge
1,2008-08-11,1,b why wont america and nato help us if they wont help us now why did we help them in iraq,b bush puts foot down on georgian conflict,b jewish georgian minister thanks to israeli training we re fending off russia,b georgian army flees in disarray as russians advance gori abandoned to russia without a shot fired,b olympic opening ceremony fireworks faked,b what were the mossad with fraudulent new zealand passports doing in iraq,b russia angered by israeli military sale to georgia,b an american citizen living in s ossetia blames u s and georgian leaders for the genocide of innocent people,...,b israel and the us behind the georgian aggression,b do not believe tv neither russian nor georgian there are much more victims,b riots are still going on in montreal canada because police murdered a boy on saturday,b china to overtake us as largest manufacturer,b war in south ossetia pics,b israeli physicians group condemns state torture,b russia has just beaten the united states over the head with peak oil,b perhaps the question about the georgia russia conflict,b russia is so much better at war,b so this is what it s come to trading sex for food
2,2008-08-12,0,b remember that adorable 9 year old who sang at the opening ceremonies that was fake too,b russia ends georgia operation,b if we had no sexual harassment we would have no children,b al qa eda is losing support in iraq because of a brutal crackdown on activities it regards as un islamic including women buying cucumbers,b ceasefire in georgia putin outmaneuvers the west,b why microsoft and intel tried to kill the xo 100 laptop,b stratfor the russo georgian war and the balance of power,b i m trying to get a sense of this whole georgia russia war vote up if you think georgia started it or down if you think russia did,...,b u s troops still in georgia did you know they were in georgia in the first place,b why russias response to georgia was right,b gorbachev accuses u s of making a serious blunder in pursuing its interest in the caucasus region,b russia georgia and nato cold war two,b remember that adorable 62 year old who led your country into war based on evidence that was fake too,b wa

In [37]:
data3 = data2.drop(axis = 0, labels = [277,384,681])

In [88]:
data3.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,b georgia downs two russian warplanes as countries move to brink of war,b breaking musharraf to be impeached,b russia today columns of troops roll into south ossetia footage from fighting youtube,b russian tanks are moving towards the capital of south ossetia which has reportedly been completely destroyed by georgian artillery fire,b afghan children raped with impunity u n official says this is sick a three year old was raped and they do nothing,b 150 russian tanks have entered south ossetia whilst georgia shoots down two russian jets,b breaking georgia invades south ossetia russia warned it would intervene on so s side,b the enemy combatent trials are nothing but a sham salim haman has been sentenced to 5 1 2 years but will be kept longer anyway just because they feel like it,...,b georgia invades south ossetia if russia gets involved will nato absorb georgia and unleash a full scale war,b al qaeda faces islamist backlash,b condoleezza rice the us would not act to prevent an israeli strike on iran israeli defense minister ehud barak israel is prepared for uncompromising victory in the case of military hostilities,b this is a busy day the european union has approved new sanctions against iran in protest at its nuclear programme,b georgia will withdraw 1 000 soldiers from iraq to help fight off russian forces in georgia s breakaway region of south ossetia,b why the pentagon thinks attacking iran is a bad idea us news amp world report,b caucasus in crisis georgia invades south ossetia,b indian shoe manufactory and again in a series of you do not like your work,b visitors suffering from mental illnesses banned from olympics,b no help for mexico s kidnapping surge
1,2008-08-11,1,b why wont america and nato help us if they wont help us now why did we help them in iraq,b bush puts foot down on georgian conflict,b jewish georgian minister thanks to israeli training we re fending off russia,b georgian army flees in disarray as russians advance gori abandoned to russia without a shot fired,b olympic opening ceremony fireworks faked,b what were the mossad with fraudulent new zealand passports doing in iraq,b russia angered by israeli military sale to georgia,b an american citizen living in s ossetia blames u s and georgian leaders for the genocide of innocent people,...,b israel and the us behind the georgian aggression,b do not believe tv neither russian nor georgian there are much more victims,b riots are still going on in montreal canada because police murdered a boy on saturday,b china to overtake us as largest manufacturer,b war in south ossetia pics,b israeli physicians group condemns state torture,b russia has just beaten the united states over the head with peak oil,b perhaps the question about the georgia russia conflict,b russia is so much better at war,b so this is what it s come to trading sex for food
2,2008-08-12,0,b remember that adorable 9 year old who sang at the opening ceremonies that was fake too,b russia ends georgia operation,b if we had no sexual harassment we would have no children,b al qa eda is losing support in iraq because of a brutal crackdown on activities it regards as un islamic including women buying cucumbers,b ceasefire in georgia putin outmaneuvers the west,b why microsoft and intel tried to kill the xo 100 laptop,b stratfor the russo georgian war and the balance of power,b i m trying to get a sense of this whole georgia russia war vote up if you think georgia started it or down if you think russia did,...,b u s troops still in georgia did you know they were in georgia in the first place,b why russias response to georgia was right,b gorbachev accuses u s of making a serious blunder in pursuing its interest in the caucasus region,b russia georgia and nato cold war two,b remember that adorable 62 year old who led your country into war based on evidence that was fake too,b wa

In [89]:
def df_concat_str_columns(df, columns):
    concat_columns = df['Top1'].copy()
    for row_index, row_value in df.iterrows():
        single_string = ""
        #print()
        for col in columns:
            single_string += " " + str(df[col].loc[row_index])
        concat_columns[row_index] = single_string
    return concat_columns

columns = data3.columns.values[2:]
data4 = df_concat_str_columns(data3, columns)
data4 = pd.DataFrame(data4)
data4.head()

,Top1
0,b georgia downs two russian warplanes as countries move to brink of war b breaking musharraf to be impeached b russia today columns of troops roll into south ossetia footage from fighting youtube b russian tanks are moving towards the capital of south ossetia which has reportedly been completely destroyed by georgian artillery fire b afghan children raped with impunity u n official says this is sick a three year old was raped and they do nothing b 150 russian tanks have entered south ossetia whilst georgia shoots down two russian jets b breaking georgia invades south ossetia russia warned it would intervene on so s side b the enemy combatent trials are nothing but a sham salim haman has been sentenced to 5 1 2 years but will be kept longer anyway just because they feel like it b georgian troops retreat from s osettain capital presumably leaving several hundred people killed video b did the u s prep georgia for war with russia b rice gives green light for israel to attack iran says u s has no veto over israeli military ops b announcing class action lawsuit on behalf of american public against the fbi b so russia and georgia are at war and the nyt s top story is opening ceremonies of the olympics what a fucking disgrace and yet further proof of the decline of journalism b china tells bush to stay out of other countries affairs b did world war iii start today b georgia invades south ossetia if russia gets involved will nato absorb georgia and unleash a full scale war b al qaeda faces islamist backlash b condoleezza rice the us would not act to prevent an israeli strike on iran israeli defense minister ehud barak israel is prepared for uncompromising victory in the case of military hostilities b this is a busy day the european union has approved new sanctions against iran in protest at its nuclear programme b georgia will withdraw 1 000 soldiers from iraq to help fight off russian forces in georgia s breakaway region of south ossetia b why the pentagon thinks attacking iran is a bad idea us news amp world report b caucasus in crisis georgia invades south ossetia b indian shoe manufactory and again in a series of you do not like your work b visitors suffering from mental illnesses banned from olympics b no help for mexico s kidnapping surge
1,b why wont america and nato help us if they wont help us now why did we help them in iraq b bush puts foot down on georgian conflict b jewish georgian minister thanks to israeli training we re fending off russia b georgian army flees in disarray as russians advance gori abandoned to russia without a shot fired b olympic opening ceremony fireworks faked b what were the mossad with fraudulent new zealand passports doing in iraq b russia angered by israeli military sale to georgia b an american citizen living in s ossetia blames u s and georgian leaders for the genocide of innocent people b welcome to world war iv now in high definition b georgia s move a mistake of monumental proportions b russia presses deeper into georgia u s says regime change is goal b abhinav bindra wins first ever individual olympic gold medal for india b u s ship heads for arctic to define territory b drivers in a jerusalem taxi station threaten to quit rather than work for their new boss an arab b the french team is stunned by phelps and the 4x100m relay team b israel and the us behind the georgian aggression b do not believe tv neither russian nor georgian there are much more victims b riots are still going on in montreal canada because police murdered a boy on saturday b china to overtake us as largest manufacturer b war in south ossetia pics b israeli physicians group condemns state torture b russia has just beaten the united states over the head with peak oil b perhaps the question about the georgia russia conflict b russia is so much better at war b so this is what it s come to trading sex for food
2,b remember that adorable 9 year old who sang at the opening ceremonies that was fake too b russia ends georgi

In [90]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sw = set(stopwords.words("english"))
print(sw)

{'both', 'i', 'can', 'these', 'is', 'he', 'own', 'what', 'will', "you're", "you've", 'your', 'those', 'do', 'himself', 'them', "hasn't", 'having', 'during', 'that', 'yourselves', 'its', 'again', "couldn't", 'her', 'under', 'too', 'there', 'through', 'isn', 's', 'when', 'haven', 'd', 'but', "shouldn't", 'if', 'are', 'very', 't', 'hasn', "needn't", 'down', 'this', "won't", "should've", 'before', 'than', 'on', 'my', 'did', 'up', 'here', 'whom', 'how', 'against', 'the', 'with', 've', "haven't", 'themselves', "you'd", 'we', 'some', "shan't", "you'll", 're', 'which', 'and', 'so', 'an', 'over', 'their', "mustn't", 'nor', 'not', 'off', 'same', "didn't", "it's", 'in', "mightn't", 'then', 'once', 'been', 'it', 'wasn', "doesn't", 'between', 'as', 'doesn', 'theirs', "hadn't", "wasn't", 'because', 'yourself', 'to', 'had', 'o', "wouldn't", 'who', 'she', 'don', 'wouldn', 'll', 'other', 'didn', 'into', 'was', 'more', 'mustn', 'no', 'yours', 'aren', 'while', 'further', 'mightn', 'at', 'or', 'weren', 'h

In [170]:
data5 = data4['Top1'].apply(word_tokenize)
data5 = pd.DataFrame(data5)
data5.head()

,Top1
0,"[b, georgia, downs, two, russian, warplanes, as, countries, move, to, brink, of, war, b, breaking, musharraf, to, be, impeached, b, russia, today, columns, of, troops, roll, into, south, ossetia, footage, from, fighting, youtube, b, russian, tanks, are, moving, towards, the, capital, of, south, ossetia, which, has, reportedly, been, completely, destroyed, by, georgian, artillery, fire, b, afghan, children, raped, with, impunity, u, n, official, says, this, is, sick, a, three, year, old, was, raped, and, they, do, nothing, b, 150, russian, tanks, have, entered, south, ossetia, whilst, georgia, shoots, down, two, russian, jets, b, breaking, georgia, invades, south, ossetia, russia, warned, ...]"
1,"[b, why, wont, america, and, nato, help, us, if, they, wont, help, us, now, why, did, we, help, them, in, iraq, b, bush, puts, foot, down, on, georgian, conflict, b, jewish, georgian, minister, thanks, to, israeli, training, we, re, fending, off, russia, b, georgian, army, flees, in, disarray, as, russians, advance, gori, abandoned, to, russia, without, a, shot, fired, b, olympic, opening, ceremony, fireworks, faked, b, what, were, the, mossad, with, fraudulent, new, zealand, passports, doing, in, iraq, b, russia, angered, by, israeli, military, sale, to, georgia, b, an, american, citizen, living, in, s, ossetia, blames, u, s, and, georgian, ...]"
2,"[b, remember, that, adorable, 9, year, old, who, sang, at, the, opening, ceremonies, that, was, fake, too, b, russia, ends, georgia, operation, b, if, we, had, no, sexual, harassment, we, would, have, no, children, b, al, qa, eda, is, losing, support, in, iraq, because, of, a, brutal, crackdown, on, activities, it, regards, as, un, islamic, including, women, buying, cucumbers, b, ceasefire, in, georgia, putin, outmaneuvers, the, west, b, why, microsoft, and, intel, tried, to, kill, the, xo, 100, laptop, b, stratfor, the, russo, georgian, war, and, the, balance, of, power, b, i, m, trying, to, get, a, sense, of, this, ...]"
3,"[b, u, s, refuses, israel, weapons, to, attack, iran, report, b, when, the, president, ordered, to, attack, tskhinvali, the, capital, of, south, ossetia, we, knew, then, we, were, doomed, how, come, he, didn, t, realize, that, b, israel, clears, troops, who, killed, reuters, cameraman, b, britain, s, policy, of, being, tough, on, drugs, is, pointless, says, a, former, civil, servant, who, once, ran, the, cabinet, s, anti, drugs, unit, b, body, of, 14, year, old, found, in, trunk, latest, ransom, paid, kidnapping, victim, in, mexico, head, cop, quits, prez, dissolves, suspect, elite, task, force, b, china, has, moved, 10, million, ...]"
4,"[b, all, the, experts, admit, that, we, should, legalise, drugs, b, war, in, south, osetia, 89, pictures, made, by, a, russian, soldier, b, swedish, wrestler, ara, abrahamian, throws, away, medal, in, olympic, hissy, fit, b, russia, exaggerated, the, death, toll, in, south, ossetia, now, only, 44, were, originally, killed, compared, to, 2, 000, b, missile, that, killed, 9, inside, pakistan, may, have, been, launched, by, the, cia, b, rushdie, condemns, random, house, s, refusal, to, publish, novel, for, fear, of, muslim, retaliation, b, poland, and, us, agree, to, missle, defense, deal, interesting, timing, b, will, the, russians, conquer, tblisi, bet, ...]"


In [159]:
for i, row in data5.iterrows():
    #print(row['Top1'])
    row['Top1'] = [w for w in row['Top1'] if w not in sw]

In [160]:
data5.head()

,Top1
0,"[b, georgia, downs, two, russian, warplanes, countries, move, brink, war, b, breaking, musharraf, impeached, b, russia, today, columns, troops, roll, south, ossetia, footage, fighting, youtube, b, russian, tanks, moving, towards, capital, south, ossetia, reportedly, completely, destroyed, georgian, artillery, fire, b, afghan, children, raped, impunity, u, n, official, says, sick, three, year, old, raped, nothing, b, 150, russian, tanks, entered, south, ossetia, whilst, georgia, shoots, two, russian, jets, b, breaking, georgia, invades, south, ossetia, russia, warned, would, intervene, side, b, enemy, combatent, trials, nothing, sham, salim, haman, sentenced, 5, 1, 2, years, kept, longer, anyway, feel, like, b, georgian, troops, retreat, ...]"
1,"[b, wont, america, nato, help, us, wont, help, us, help, iraq, b, bush, puts, foot, georgian, conflict, b, jewish, georgian, minister, thanks, israeli, training, fending, russia, b, georgian, army, flees, disarray, russians, advance, gori, abandoned, russia, without, shot, fired, b, olympic, opening, ceremony, fireworks, faked, b, mossad, fraudulent, new, zealand, passports, iraq, b, russia, angered, israeli, military, sale, georgia, b, american, citizen, living, ossetia, blames, u, georgian, leaders, genocide, innocent, people, b, welcome, world, war, iv, high, definition, b, georgia, move, mistake, monumental, proportions, b, russia, presses, deeper, georgia, u, says, regime, change, goal, b, abhinav, bindra, wins, first, ever, ...]"
2,"[b, remember, adorable, 9, year, old, sang, opening, ceremonies, fake, b, russia, ends, georgia, operation, b, sexual, harassment, would, children, b, al, qa, eda, losing, support, iraq, brutal, crackdown, activities, regards, un, islamic, including, women, buying, cucumbers, b, ceasefire, georgia, putin, outmaneuvers, west, b, microsoft, intel, tried, kill, xo, 100, laptop, b, stratfor, russo, georgian, war, balance, power, b, trying, get, sense, whole, georgia, russia, war, vote, think, georgia, started, think, russia, b, us, military, surprised, timing, swiftness, russian, military, move, south, ossetia, still, trying, sort, happened, us, defense, official, said, monday, b, u, beats, war, drum, iran, dumps, dollar, ...]"
3,"[b, u, refuses, israel, weapons, attack, iran, report, b, president, ordered, attack, tskhinvali, capital, south, ossetia, knew, doomed, come, realize, b, israel, clears, troops, killed, reuters, cameraman, b, britain, policy, tough, drugs, pointless, says, former, civil, servant, ran, cabinet, anti, drugs, unit, b, body, 14, year, old, found, trunk, latest, ransom, paid, kidnapping, victim, mexico, head, cop, quits, prez, dissolves, suspect, elite, task, force, b, china, moved, 10, million, quake, survivors, prefab, homes, b, bush, announces, operation, get, russia, grill, yeah, end, well, b, russian, forces, sink, georgian, ships, b, commander, navy, air, reconnaissance, squadron, provides, president, defense, secretary, airborne, ...]"
4,"[b, experts, admit, legalise, drugs, b, war, south, osetia, 89, pictures, made, russian, soldier, b, swedish, wrestler, ara, abrahamian, throws, away, medal, olympic, hissy, fit, b, russia, exaggerated, death, toll, south, ossetia, 44, originally, killed, compared, 2, 000, b, missile, killed, 9, inside, pakistan, may, launched, cia, b, rushdie, condemns, random, house, refusal, publish, novel, fear, muslim, retaliation, b, poland, us, agree, missle, defense, deal, interesting, timing, b, russians, conquer, tblisi, bet, seriously, bet, b, russia, exaggerating, south, ossetian, death, toll, says, human, rights, group, b, musharraf, expected, resign, rather, face, impeachment, b, moscow, made, plans, months, ago, invade, georgia, ...]"


In [161]:
from nltk import pos_tag

pos_tags = []
for i, row in data5.iterrows():
    pos_tags.append(pos_tag(row['Top1']))

In [167]:
import nltk

dist = nltk.FreqDist([pos[1] for pos in pos_tags[0][:]])
dist

FreqDist({'NN': 108, 'JJ': 61, 'NNS': 28, 'VBP': 14, 'VBD': 12, 'CD': 10, 'VBG': 10, 'VBZ': 8, 'RB': 7, 'IN': 6, ...})

In [172]:
import tqdm as tqdm

lengths = []

for i, row in data5.iterrows():
    lengths.append(len(data5.loc[i, 'Top1']))
lengths = np.array(lengths)

for i, row in data5.iterrows():
    print(i)
    dist = nltk.FreqDist([pos[1] for pos in pos_tags[i][:]])
    
    for k, v in dist.items():
        data5.loc[i, k] = v / lengths[i]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986


IndexError: list index out of range

In [173]:
data5.head()

,Top1,NN,NNS,CD,JJ,VBP,VBG,VBD,RB,VBZ,...,WP,NNPS,WDT,WRB,PRP$,TO,POS,PDT,'',EX
0,"[b, georgia, downs, two, russian, warplanes, as, countries, move, to, brink, of, war, b, breaking, musharraf, to, be, impeached, b, russia, today, columns, of, troops, roll, into, south, ossetia, footage, from, fighting, youtube, b, russian, tanks, are, moving, towards, the, capital, of, south, ossetia, which, has, reportedly, been, completely, destroyed, by, georgian, artillery, fire, b, afghan, children, raped, with, impunity, u, n, official, says, this, is, sick, a, three, year, old, was, raped, and, they, do, nothing, b, 150, russian, tanks, have, entered, south, ossetia, whilst, georgia, shoots, down, two, russian, jets, b, breaking, georgia, invades, south, ossetia, russia, warned, ...]",0.265356,0.068796,0.024570,0.149877,0.034398,0.024570,0.029484,0.017199,0.019656,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[b, why, wont, america, and, nato, help, us, if, they, wont, help, us, now, why, did, we, help, them, in, iraq, b, bush, puts, foot, down, on, georgian, conflict, b, jewish, georgian, minister, thanks, to, israeli, training, we, re, fending, off, russia, b, georgian, army, flees, in, disarray, as, russians, advance, gori, abandoned, to, russia, without, a, shot, fired, b, olympic, opening, ceremony, fireworks, faked, b, what, were, the, mossad, with, fraudulent, new, zealand, passports, doing, in, iraq, b, russia, angered, by, israeli, military, sale, to, georgia, b, an, american, citizen, living, in, s, ossetia, blames, u, s, and, georgian, ...]",0.272727,0.063973,0.003367,0.151515,0.047138,0.016835,0.026936,0.016835,0.013468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[b, remember, that, adorable, 9, year, old, who, sang, at, the, opening, ceremonies, that, was, fake, too, b, russia, ends, georgia, operation, b, if, we, had, no, sexual, harassment, we, would, have, no, children, b, al, qa, eda, is, losing, support, in, iraq, because, of, a, brutal, crackdown, on, activities, it, regards, as, un, islamic, including, women, buying, cucumbers, b, ceasefire, in, georgia, putin, outmaneuvers, the, west, b, why, microsoft, and, intel, tried, to, kill, the, xo, 100, laptop, b, stratfor, the, russo, georgian, war, and, the, balance, of, power, b, i, m, trying, to, get, a, sense, of, this, ...]",0.289406,0.049096,0.018088,0.118863,0.036176,0.033592,0.020672,0.018088,0.007752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[b, u, s, refuses, israel, weapons, to, attack, iran, report, b, when, the, president, ordered, to, attack, tskhinvali, the, capital, of, south, ossetia, we, knew, then, we, were, doomed, how, come, he, didn, t, realize, that, b, israel, clears, troops, who, killed, reuters, cameraman, b, britain, s, policy, of, being, tough, on, drugs, is, pointless, says, a, former, civil, servant, who, once, ran, the, cabinet, s, anti, drugs, unit, b, body, of, 14, year, old, found, in, trunk, latest, ransom, paid, kidnapping, victim, in, mexico, head, cop, quits, prez, dissolves, suspect, elite, task, force, b, china, has, moved, 10, million, ...]",0.284884,0.093023,0.020349,0.133721,0.058140,0.017442,0.029070,0.008721,0.020349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[b, all, the, experts, admit, that, we, should, legalise, drugs, b, war, in, south, osetia, 89, pictures, made, by, a, russian, soldier, b, swedish, wrestler, ara, abrahamian, throws, away, medal, in, olympic, hissy, fit, b, russia, exaggerated, the, death, toll, in, south, ossetia, now, only, 44, were, originally, killed, compared, to, 2, 000, b, missile, that, killed, 9, inside, pakistan, may, have, been, launched, by, the, cia, b, rushdie, condemns, random, house, s, refusal, to, publish, novel, for, fear, of, muslim, retaliation, b, poland, and, us, agree, to, missle, defense, deal, interesting, timing, b, will, the, russians, conquer, tblisi, bet, ...]",0.321311,0.072131,0.019672,0.131148,0.062295,0.013115,0.022951,0.032787,0.006557,...,NaN,NaN,NaN,NaN,NaN

In [174]:
data5['Label'] = data3['Label']

In [175]:
data5.head()

,Top1,NN,NNS,CD,JJ,VBP,VBG,VBD,RB,VBZ,...,NNPS,WDT,WRB,PRP$,TO,POS,PDT,'',EX,Label
0,"[b, georgia, downs, two, russian, warplanes, as, countries, move, to, brink, of, war, b, breaking, musharraf, to, be, impeached, b, russia, today, columns, of, troops, roll, into, south, ossetia, footage, from, fighting, youtube, b, russian, tanks, are, moving, towards, the, capital, of, south, ossetia, which, has, reportedly, been, completely, destroyed, by, georgian, artillery, fire, b, afghan, children, raped, with, impunity, u, n, official, says, this, is, sick, a, three, year, old, was, raped, and, they, do, nothing, b, 150, russian, tanks, have, entered, south, ossetia, whilst, georgia, shoots, down, two, russian, jets, b, breaking, georgia, invades, south, ossetia, russia, warned, ...]",0.265356,0.068796,0.024570,0.149877,0.034398,0.024570,0.029484,0.017199,0.019656,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,"[b, why, wont, america, and, nato, help, us, if, they, wont, help, us, now, why, did, we, help, them, in, iraq, b, bush, puts, foot, down, on, georgian, conflict, b, jewish, georgian, minister, thanks, to, israeli, training, we, re, fending, off, russia, b, georgian, army, flees, in, disarray, as, russians, advance, gori, abandoned, to, russia, without, a, shot, fired, b, olympic, opening, ceremony, fireworks, faked, b, what, were, the, mossad, with, fraudulent, new, zealand, passports, doing, in, iraq, b, russia, angered, by, israeli, military, sale, to, georgia, b, an, american, citizen, living, in, s, ossetia, blames, u, s, and, georgian, ...]",0.272727,0.063973,0.003367,0.151515,0.047138,0.016835,0.026936,0.016835,0.013468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"[b, remember, that, adorable, 9, year, old, who, sang, at, the, opening, ceremonies, that, was, fake, too, b, russia, ends, georgia, operation, b, if, we, had, no, sexual, harassment, we, would, have, no, children, b, al, qa, eda, is, losing, support, in, iraq, because, of, a, brutal, crackdown, on, activities, it, regards, as, un, islamic, including, women, buying, cucumbers, b, ceasefire, in, georgia, putin, outmaneuvers, the, west, b, why, microsoft, and, intel, tried, to, kill, the, xo, 100, laptop, b, stratfor, the, russo, georgian, war, and, the, balance, of, power, b, i, m, trying, to, get, a, sense, of, this, ...]",0.289406,0.049096,0.018088,0.118863,0.036176,0.033592,0.020672,0.018088,0.007752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,"[b, u, s, refuses, israel, weapons, to, attack, iran, report, b, when, the, president, ordered, to, attack, tskhinvali, the, capital, of, south, ossetia, we, knew, then, we, were, doomed, how, come, he, didn, t, realize, that, b, israel, clears, troops, who, killed, reuters, cameraman, b, britain, s, policy, of, being, tough, on, drugs, is, pointless, says, a, former, civil, servant, who, once, ran, the, cabinet, s, anti, drugs, unit, b, body, of, 14, year, old, found, in, trunk, latest, ransom, paid, kidnapping, victim, in, mexico, head, cop, quits, prez, dissolves, suspect, elite, task, force, b, china, has, moved, 10, million, ...]",0.284884,0.093023,0.020349,0.133721,0.058140,0.017442,0.029070,0.008721,0.020349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,"[b, all, the, experts, admit, that, we, should, legalise, drugs, b, war, in, south, osetia, 89, pictures, made, by, a, russian, soldier, b, swedish, wrestler, ara, abrahamian, throws, away, medal, in, olympic, hissy, fit, b, russia, exaggerated, the, death, toll, in, south, ossetia, now, only, 44, were, originally, killed, compared, to, 2, 000, b, missile, that, killed, 9, inside, pakistan, may, have, been, launched, by, the, cia, b, rushdie, condemns, random, house, s, refusal, to, publish, novel, for, fear, of, muslim, retaliation, b, poland, and, us, agree, to, missle, defense, deal, interesting, timing, b, will, the, russians, conquer, tblisi, bet, ...]",0.321311,0.072131,0.019672,0.131148,0.062295,0.013115,0.022951,0.032787,0.006557,...,NaN,NaN,NaN,NaN,NaN,NaN,

In [186]:
data5 = data5.fillna(0)

In [187]:
df = data5.copy()

In [188]:
df.iloc[:,1:-2]

,NN,NNS,CD,JJ,VBP,VBG,VBD,RB,VBZ,MD,...,RBR,WP,NNPS,WDT,WRB,PRP$,TO,POS,PDT,''
0,0.265356,0.068796,0.024570,0.149877,0.034398,0.024570,0.029484,0.017199,0.019656,0.004914,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.272727,0.063973,0.003367,0.151515,0.047138,0.016835,0.026936,0.016835,0.013468,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.289406,0.049096,0.018088,0.118863,0.036176,0.033592,0.020672,0.018088,0.007752,0.005168,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.284884,0.093023,0.020349,0.133721,0.058140,0.017442,0.029070,0.008721,0.020349,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.321311,0.072131,0.019672,0.131148,0.062295,0.013115,0.022951,0.032787,0.006557,0.006557,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,0.221053,0.122105,0.037895,0.109474,0.033684,0.018947,0.042105,0.012632,0.023158,0.006316,...,0.006316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1985,0.220183,0.100917,0.031193,0.130275,0.029358,0.016514,0.034862,0.020183,0.027523,0.003670,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1986,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [191]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:,1:-2],
    df.iloc[:,-1],
    train_size = 0.80,
    random_state = 1234
)
display(x_train)
display(x_test)

,NN,NNS,CD,JJ,VBP,VBG,VBD,RB,VBZ,MD,...,RBR,WP,NNPS,WDT,WRB,PRP$,TO,POS,PDT,''
1889,0.239411,0.086556,0.029466,0.134438,0.033149,0.034991,0.023941,0.014733,0.014733,0.003683,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
0,0.265356,0.068796,0.024570,0.149877,0.034398,0.024570,0.029484,0.017199,0.019656,0.004914,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
814,0.242366,0.104962,0.030534,0.085878,0.032443,0.038168,0.032443,0.020992,0.020992,0.000000,...,0.000000,0.000000,0.0,0.001908,0.0,0.0,0.0,0.0,0.0,0.000000
1208,0.195402,0.122605,0.013410,0.151341,0.053640,0.022989,0.034483,0.013410,0.007663,0.003831,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
282,0.261157,0.077686,0.018182,0.114050,0.036364,0.014876,0.029752,0.021488,0.019835,0.003306,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,0.201869,0.093458,0.011215,0.155140,0.039252,0.028037,0.024299,0.018692,0.016822,0.005607,...,0.000000,0.001869,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001869
1080,0.220310,0.089501,0.022375,0.139415,0.029260,0.043029,0.037866,0.024096,0.013769,0.003442,...,0.001721,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1321,0.211813,0.107943,0.030550,0.152749,0.040733,0.030550,0.040733,0.020367,0.024440,0.002037,...,0.006110,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
726,0.207547,0.091644,0.018868,0.153639,0.051213,0.029650,0.018868,0.018868,0.029650,0.005391,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


,NN,NNS,CD,JJ,VBP,VBG,VBD,RB,VBZ,MD,...,RBR,WP,NNPS,WDT,WRB,PRP$,TO,POS,PDT,''
1057,0.223636,0.116364,0.029091,0.112727,0.052727,0.016364,0.038182,0.023636,0.020000,0.003636,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
742,0.202811,0.096386,0.028112,0.124498,0.032129,0.028112,0.028112,0.050201,0.014056,0.004016,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
240,0.305128,0.076923,0.028205,0.105128,0.025641,0.012821,0.053846,0.007692,0.020513,0.000000,...,0.005128,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
188,0.257353,0.098039,0.026961,0.117647,0.026961,0.022059,0.039216,0.009804,0.012255,0.004902,...,0.002451,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1248,0.228346,0.088583,0.035433,0.129921,0.035433,0.033465,0.031496,0.023622,0.017717,0.001969,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,0.265060,0.098795,0.028916,0.149398,0.040964,0.021687,0.045783,0.016867,0.012048,0.000000,...,0.002410,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1972,0.229268,0.107317,0.026829,0.126829,0.041463,0.046341,0.026829,0.029268,0.019512,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1367,0.235669,0.089172,0.027601,0.142251,0.038217,0.023355,0.027601,0.014862,0.023355,0.002123,...,0.002123,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1162,0.211155,0.091633,0.017928,0.105578,0.031873,0.019920,0.035857,0.015936,0.017928,0.009960,...,0.000000,0.0,0.0,0.001992,0.0,0.0,0.0,0.0,0.0,0.0


Naive Bayes

In [195]:
def make_percent(num):
    return '%.2f'%(num * 100) + '%'

from sklearn.naive_bayes import GaussianNB
gnbc = GaussianNB()
start = time.time()
gnbc.fit(x_train, y_train)
stop = time.time()
print(make_percent(gnbc.score(x_test, y_test)), '|| ' + '%.2f'%(stop - start) + ' seconds')

53.27% || 0.00 seconds


Decision Tree

In [212]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
start = time.time()
dtc.fit(x_train, y_train)
preds2 = dtc.predict(x_test)
stop = time.time()
print(make_percent(dtc.score(x_test, y_test)), '|| ' + '%.2f'%(stop - start) + ' seconds')

51.51% || 0.04 seconds


In [213]:
confusion_matrix(preds2, y_test)

array([[ 88,  99],
       [ 94, 117]])

Logistic Regression

In [210]:
from sklearn.linear_model import LogisticRegression
lgc = LogisticRegression()
start = time.time()
lgc.fit(x_train, y_train)
preds = lgc.predict(x_test)
stop = time.time()
print(make_percent(lgc.score(x_test, y_test)), '|| ' + '%.2f'%(stop - start) + ' seconds')

54.27% || 0.01 seconds


/home/sa/Documents/env3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [211]:
from sklearn.metrics import confusion_matrix
confusion_matrix(preds, y_test)

array([[  0,   0],
       [182, 216]])

Random Forest

In [199]:
from sklearn.ensemble import RandomForestClassifier
rfc_gini = RandomForestClassifier(n_estimators = 100, criterion = 'gini')
start = time.time()
rfc_gini.fit(x_train, y_train)
stop = time.time()
print(make_percent(rfc_gini.score(x_test, y_test)), '|| ' + '%.2f'%(stop - start) + ' seconds')

49.50% || 0.43 seconds


In [200]:
rfc_gini = RandomForestClassifier(n_estimators = 1000, criterion = 'gini')
start = time.time()
rfc_gini.fit(x_train, y_train)
stop = time.time()
print(make_percent(rfc_gini.score(x_test, y_test)), '|| ' + '%.2f'%(stop - start) + ' seconds')

51.51% || 3.84 seconds


In [201]:
rfc_gini = RandomForestClassifier(n_estimators = 100, criterion = 'gini', min_samples_leaf = 200)
start = time.time()
rfc_gini.fit(x_train, y_train)
stop = time.time()
print(make_percent(rfc_gini.score(x_test, y_test)), '|| ' + '%.2f'%(stop - start) + ' seconds')

53.52% || 0.13 seconds


Random Forest (Information Gain)

In [202]:
rfc_info = RandomForestClassifier(n_estimators = 100, criterion = 'entropy')
start = time.time()
rfc_info.fit(x_train, y_train)
stop = time.time()
print(make_percent(rfc_info.score(x_test, y_test)), '|| ' + '%.2f'%(stop - start) + ' seconds')

52.76% || 0.73 seconds


Extremely Randomized Trees

In [203]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators = 100, criterion = 'gini')
start = time.time()
etc.fit(x_train, y_train)
stop = time.time()
print(make_percent(etc.score(x_test, y_test)), '|| ' + '%.2f'%(stop - start) + ' seconds')

53.27% || 0.26 seconds


ADA Boost

In [204]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier()
start = time.time()
abc.fit(x_train, y_train)
stop = time.time()
print(make_percent(abc.score(x_test, y_test)), '|| ' + '%.2f'%(stop - start) + ' seconds')

47.74% || 0.19 seconds
